In [12]:
## Function for Extracting Top drugs
import joblib
import pandas as pd
from tensorflow.keras.models import load_model
import tensorflow as tf
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np


# Load the TF-IDF vectorizer
tfidf_vectorizer2 = joblib.load('tfidf_vectorizer2.pkl')

# Load the DNN model
condition_predictor = load_model('condition_predictor_dnn.h5')

# Load df_top for top drugs
df_top = joblib.load('df_top.pkl')

def top_drugs_extractor(condition):
    drug_lst = df_top[df_top['condition'] == condition]['drugName'].head(3).tolist()
    return drug_lst

def predict_text(lst_text):
    df_test = pd.DataFrame(lst_text, columns=['test_sent'])
    df_test["test_sent"] = df_test["test_sent"].apply(review_to_words)
    tfidf_bigram = tfidf_vectorizer2.transform(lst_text)
    prediction = condition_predictor.predict(tfidf_bigram)
    df_test['prediction'] = prediction
    return df_test

# Function to get input sentences from the user
def get_sentences():
    sentences = []
    print("Enter your sentences one by one. Press Enter on an empty line to stop.")

    while True:
        sentence = input("Enter a sentence: ")
        if sentence.strip() == "":
            break
        sentences.append(sentence)

    return sentences

# Get sentences from the user
sentences = get_sentences()

# Transform the sentences using the loaded TF-IDF vectorizer
tfidf_trigram = tfidf_vectorizer2.transform(sentences)

# Convert the SciPy sparse matrix to a TensorFlow SparseTensor
tfidf_sparse_tensor = tf.sparse.SparseTensor(
    indices=np.array(tfidf_trigram.nonzero()).T,
    values=tfidf_trigram.data,
    dense_shape=tfidf_trigram.shape
)

# Reorder the sparse tensor
tfidf_sparse_tensor_ordered = tf.sparse.reorder(tfidf_sparse_tensor)

# Convert the sparse tensor to a dense format if necessary
tfidf_dense = tf.sparse.to_dense(tfidf_sparse_tensor_ordered)

# Predict the conditions
predictions = condition_predictor.predict(tfidf_dense)

# Assuming the model's output is a one-hot encoded array, get the predicted class index
predicted_indices = predictions.argmax(axis=1)

# Define the mapping of class indices to condition names
index_to_condition = {0: "High Blood Pressure", 1: "Depression", 2: "Diabetes, Type 2", 3: "Birth Control"}

# Convert predicted indices to condition names
predicted_labels = [index_to_condition[index] for index in predicted_indices]

# Loop through predictions and print the top 3 suggested drugs for each condition
for text, label in zip(sentences, predicted_labels):
    top_drugs = top_drugs_extractor(label)
    print(f"text: {text}\nCondition: {label}")
    print("Top 3 Suggested Drugs:")
    for drug in top_drugs:
        print(drug)
    print()

Enter your sentences one by one. Press Enter on an empty line to stop.


Enter a sentence:  "This is the third med I&#039;ve tried for anxiety and mild depression. Been on it for a week and I hate it so much. I am so dizzy, I have major diarrhea and feel worse than I started. Contacting my doc in the am and changing asap."
Enter a sentence:  


1/1 [==============================] - 0s 67ms/step
text: "This is the third med I&#039;ve tried for anxiety and mild depression. Been on it for a week and I hate it so much. I am so dizzy, I have major diarrhea and feel worse than I started. Contacting my doc in the am and changing asap."
Condition: Depression
Top 3 Suggested Drugs:
Sertraline
Zoloft
Viibryd

